### Preamble: Keras and TensorFlow tight Integration

<img src="imgs/tweet1.png" width="50%" />


#### Suggested Reading: 

* [Launch a GPU-backed Google Compute Engine instance and setup Tensorflow, Keras and Jupyter](https://hackernoon.com/launch-a-gpu-backed-google-compute-engine-instance-and-setup-tensorflow-keras-and-jupyter-902369ed5272)

<img src="imgs/tweet2.png" width="50%" />

In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'1.2.1'

In [3]:
from tensorflow.contrib import keras

#### Tensorboard Integration

In [4]:
from keras.datasets import cifar100

(X_train, Y_train), (X_test, Y_test) = cifar100.load_data(label_mode='fine')

Using TensorFlow backend.


In [5]:
from keras import backend as K

img_rows, img_cols = 32, 32

if K.image_data_format() == 'channels_first':
    shape_ord = (3, img_rows, img_cols)
else:  # channel_last
    shape_ord = (img_rows, img_cols, 3)

In [6]:
shape_ord

(32, 32, 3)

In [7]:
X_train.shape, X_test.shape

((50000, 32, 32, 3), (10000, 32, 32, 3))

In [8]:
import numpy as np
nb_classes = len(np.unique(Y_train))

In [9]:
from keras.applications import vgg16
from keras.layers import Input

In [10]:
vgg16_model = vgg16.VGG16(weights='imagenet', include_top=False, 
                          input_tensor=Input(shape_ord))
vgg16_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0         
__________

In [11]:
for layer in vgg16_model.layers:
    layer.trainable = False  # freeze layer

In [12]:
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization

#### Keras Functional APIs! 

Remember:
**Valar Morghulis**: _All Layers are Callables!_

In [13]:
x = Flatten(input_shape=vgg16_model.output.shape)(vgg16_model.output)
x = Dense(4096, activation='relu', name='ft_fc1')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
predictions = Dense(nb_classes, activation = 'softmax')(x)

In [14]:
from keras.models import Model

In [15]:
#create graph of your new model
model = Model(inputs=vgg16_model.input, outputs=predictions)

#compile the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0         
__________

### `TensorBoard` Callback

In [17]:
from keras.callbacks import TensorBoard

```python

# Arguments
    log_dir: the path of the directory where to save the log
        files to be parsed by TensorBoard.
    histogram_freq: frequency (in epochs) at which to compute activation
        and weight histograms for the layers of the model. If set to 0,
        histograms won't be computed. Validation data (or split) must be
        specified for histogram visualizations.
    write_graph: whether to visualize the graph in TensorBoard.
        The log file can become quite large when
        write_graph is set to True.
    write_grads: whether to visualize gradient histograms in TensorBoard.
        `histogram_freq` must be greater than 0.
    write_images: whether to write model weights to visualize as
        image in TensorBoard.
    embeddings_freq: frequency (in epochs) at which selected embedding
        layers will be saved.
    embeddings_layer_names: a list of names of layers to keep eye on. If
        None or empty list all the embedding layer will be watched.
    embeddings_metadata: a dictionary which maps layer name to a file name
        in which metadata for this embedding layer is saved. 
```

See the [details](https://www.tensorflow.org/how_tos/embedding_viz/#metadata_optional)
about metadata files format. In case if the same metadata file is used for all embedding layers, string can be passed.

In [18]:
## one-hot Encoding of labels (1 to 100 classes)
from keras.utils import np_utils

In [19]:
Y_train = np_utils.to_categorical(Y_train)
Y_test = np_utils.to_categorical(Y_test)

In [20]:
Y_train.shape, Y_test.shape

((50000, 100), (10000, 100))

In [22]:
def generate_batches(X, Y, batch_size=128):
    """"""
    # Iterations has to go indefinitely
    start = 0
    while True:
        yield (X[start:start+batch_size], Y[start:start+batch_size])
        start=batch_size
        
# Get a subset of Validation Data (for Speed purposes) - 10%
X_val, Y_val = X_test[:1000], Y_test[:1000]

batch_size = 64
train_steps_per_epoch = np.floor(X_train.shape[0] / batch_size)
valid_steps_per_epoch = np.floor(X_test.shape[0] / batch_size)
model.fit_generator(generate_batches(X_train, Y_train, batch_size=batch_size), steps_per_epoch=train_steps_per_epoch, 
                    validation_data=(X_test, Y_test), epochs=10, verbose=1,
                    callbacks=[TensorBoard(log_dir='./tf_logs', histogram_freq=10, 
                                           write_graph=True, write_images=True, 
                                           embeddings_freq=10, 
                                           embeddings_layer_names=['block1_conv2', 
                                                                   'block5_conv1', 
                                                                   'ft_fc1'], 
                                           embeddings_metadata=None)])

Epoch 1/10
781/781 [==============================] - 119s - loss: 0.0157 - acc: 0.9975 - val_loss: 7.2698 - val_acc: 0.0620
Epoch 2/10
781/781 [==============================] - 55s - loss: 1.1926e-07 - acc: 1.0000 - val_loss: 7.4186 - val_acc: 0.0616
Epoch 3/10
781/781 [==============================] - 54s - loss: 1.1922e-07 - acc: 1.0000 - val_loss: 7.4713 - val_acc: 0.0623
Epoch 4/10
781/781 [==============================] - 54s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 7.4838 - val_acc: 0.0617
Epoch 5/10
781/781 [==============================] - 54s - loss: 1.1922e-07 - acc: 1.0000 - val_loss: 7.5028 - val_acc: 0.0616
Epoch 6/10
781/781 [==============================] - 54s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 7.5235 - val_acc: 0.0616
Epoch 7/10
781/781 [==============================] - 54s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 7.5361 - val_acc: 0.0612
Epoch 8/10
781/781 [==============================] - 54s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 7.53

#### Runing Tensorboard

In [23]:
%%bash
python -m tensorflow.tensorboard --logdir=./tf_logs

Process is terminated.


---